In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
a=10
def afunc():
    global a #공유변수
    a+=1 #참조해서 값을 읽어 다시 참소변수에 저장할 때는 지역변수에서만 찾는다. 단, global로 표시되어있으면 전역변수까지 확인
    print(a) # a는 afunc()입장에서는 전역변수

afunc()

11


# Shared Variables

- 모든 노드에서 사용하기 위한 공유(전역)변수: 수정 불가능
    - Broadcast Variables
- 공유변수로 지정한 값은 모든 노드에 중복되어 캐시된다.
- 반복적으로 사용해야하는 변수라면,  
  스파크의 노드는 네트워크를 통해 통신 하기 때문에 모든 노드에 중복 캐시하는 시스템적 비용보다  
  네트워크 과정에서 발생하는 오버헤드 비용이 더 많이 발생하게 된다.

## Broadcast Variables

- 각 노드에 공유되는 읽기 전용 변수

In [4]:
# 학생별 수업카테고리코드로 지정되어있는 값을 카테고리 전체이름으로 변경한다고 가정 해보자

data = [("홍길동","DE"),
    ("이제동","DS"),
    ("김철수","DE"),
    ("변현재","WD")]

code_desc = {"DE":"Data Engineer", "DS":"Data Science", "WD":"Web Developer"}
    

In [6]:
students_rdd = sc.parallelize(data,3)
students_rdd.collect()

[('홍길동', 'DE'), ('이제동', 'DS'), ('김철수', 'DE'), ('변현재', 'WD')]

In [7]:
students_rdd.mapValues(lambda e: code_desc[e]).collect()

[('홍길동', 'Data Engineer'),
 ('이제동', 'Data Science'),
 ('김철수', 'Data Engineer'),
 ('변현재', 'Web Developer')]

### 학생 전공명 변경
- 학생 data는 여러 rdd 객체로 구성되어있음
- 변경할 전공명은 code_desc 변수에 저장
- code_desc는 모든 rdd가 접근 가능해야하고, 내용이 변경되면 안됨
    - 공유변수로 등록, 변경 불가능한 readonly로 등록해서 사용할 필요가 있음: Broadcast_variables로 생성해서 사용

### Broadcast variables
- sc(spark_context)가 아닌 spark(spark_session)객체 변수로 제공
- spark.sparkContext.bradcast(변수로 사용할 값이나 변수)

In [15]:
# Broadcast_variables 사용하기
# 생성하는 시점에 등록이 되고, 객체변수도 반환됨
code_desc = {"DE":"Data Engineer", "DS":"Data Science", "WD":"Web Developer"} #등록할 변수
broadcast_S = spark.sparkContext.broadcast(code_desc) # 세션에 변수 등록하면 객체 반환
broadcast_S.value #얘가 객체라서 .value로 값에 접근해야함

# 읽기전용 변수, 수정을 하면
code_desc['DS'] = 'PROCESS'
broadcast_S.value['DS'] = 'PROCESS'
code_desc
broadcast_S.value # 여기서는 변한 것 같아 보여도 뒤에 최종 결과값 확인 할 때는 변함이 없다.

# 삭제를 해도
del(broadcast_S.value['DE'])
broadcast_S.value #action연산이므로 rdd내부에는 반영되지 않음.

students_rdd.mapValues(lambda e: code_desc[e]).collect()
students_rdd.mapValues(lambda e: broadcast_S.value[e]).collect()
# 아무일도 발생하지 않았다.
# broadcast 함수를 사용해 생성하는 시점에 이미 SparkContext에 등록



{'DE': 'Data Engineer', 'DS': 'Data Science', 'WD': 'Web Developer'}

{'DE': 'Data Engineer', 'DS': 'PROCESS', 'WD': 'Web Developer'}

{'DE': 'Data Engineer', 'DS': 'PROCESS', 'WD': 'Web Developer'}

{'DS': 'PROCESS', 'WD': 'Web Developer'}

[('홍길동', 'Data Engineer'),
 ('이제동', 'PROCESS'),
 ('김철수', 'Data Engineer'),
 ('변현재', 'Web Developer')]

[('홍길동', 'Data Engineer'),
 ('이제동', 'Data Science'),
 ('김철수', 'Data Engineer'),
 ('변현재', 'Web Developer')]

## Accumulator

- 각 노드에 공유되는 누산기 클래스
- 저장속성과 add()메소드 갖고있는 특수한 형태의 클래스
- sc.accumulator(수치형(정수형)기본값:0)
    - 각 노드에서 필요하다면 +변경은 가능함

In [16]:
accum = sc.accumulator(0) #accum.value == 0으로 초기화
rdd = sc.parallelize([1,2,3,4,5])

# accum 누산기 클래스 활용해서 rdd data를 더하는 작업
# foreach(f): 전달되는 rdd요소 각각에 대하여 f를 실행해주는 함수, f는 return값이 없어야됨(주로 accumulator에 누적저장하거나 외부시스템에 출력용도로 사용함)
rdd.foreach(lambda x: accum.add(x)) #accum.value += x
# foreach의 반환값이 없기 때문에 rdd.foreach(lambda x: accum.add(x)) # NoneType error발생: 반환값이 없어 collect불가

In [17]:
accum.value

15

### 잘못된 데이터 수 counting
- 누적연산 필요
- 누산기(accum)활용

In [18]:
# 정상 데이터: key:value
# 비정상 데이터 수 확인
accum1 = sc.accumulator(0)
rdd = sc.parallelize(["A1:V1","A2:V2","A3","A4:V4","A5;V5","A6::A6"])
                    # 형태가 잘못된 A3, 콜론대신 ; 들어간 A5;V5, 콜론두개인 A6::A6 가 비정상데이터

In [19]:
def f(e):
    global accum1
    if len(e.split(":")) != 2 :
        accum1.add(1) #accum1.value +=1 # 누산기에 누적 연산을 진행하는 함수

In [20]:
rdd.foreach(f)

In [22]:
print("잘못된 데이터 수: " + str(accum1.value))

잘못된 데이터 수: 3


In [3]:
# # accumulator를 사용하지 않는다면?
# a = 0

# # 모든 노드에서 발생하는 데이터 횟수를 확인해보자
# def change_cate(e):
#      a = a + 1
#      return broadcastStates.value[e]
    
# students_rdd.mapValues(lambda e : change_cate(e)).collect()

# # 횟수 확인
# # local variable 'a' referenced before assignment 발생
# a




In [4]:


# 모든 노드에서 발생하는 데이터 횟수를 확인해보자


# 횟수 확인
# local variable 'a' referenced before assignment 발생
